In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
#Требуют прямой порт
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.optimizers import SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/HorHum/'
test_dir = '/content/drive/MyDrive/ValidationSetHH/' 

Mounted at /content/drive


In [ ]:
img_width, img_height = 300, 300
# Размер мини-выборки
batch_size = 64
# Кол-во изображений для обучения
nb_train_samples = 1027
# Кол-во изображений для контроля
nb_test_samples = 256

In [ ]:
#Не забудем уменьшить размез изображений, чтобы не перегружать сетку через атрибут rescale (цвета закодированы в цифрах0-255 на СКЗ палетке)
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
MB = MobileNet(
    input_shape=(img_width, img_height,3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
MB.trainable = False
MB.summary()

17227776/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 150, 150, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 150, 150, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 150, 150, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 150, 150, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 150, 150, 32)      128       
_________________________________________________

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(MB)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(1, activation='sigmoid'))

#Зададим форму слоёв при постройке
model.build((None, 300, 300, 3 ))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 9, 9, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 82944)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               21233920  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 24,463,041
Trainable params: 21,234,177
Non-trainable params: 3,228,864
_________________________________________________________________


In [ ]:
# Компилируем составную сеть
model.compile(loss='binary_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [ ]:
# Обучение составной сети
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 331s 21s/step - loss: 1.4000 - accuracy: 0.7764 - val_loss: 3.5033e-04 - val_accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 70s 5s/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 69s 4s/step - loss: 0.0016 - accuracy: 0.9991 - val_loss: 1.6891e-04 - val_accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 69s 4s/step - loss: 0.0012 - accuracy: 0.9990 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 68s 4s/step - loss: 4.2830e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000


In [ ]:
# Проверяем модель на тестовой выборке
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 100.00%
